### Import libraries

In [2]:
import requests
import ssl
import pandas as pd
import numpy as np
import os

### Fetch data from API | Census 1991, 2000 e 2010

In [3]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("DEFAULT@SECLEVEL=1")
        ctx.options |= 0x4   # <-- the key part here, OP_LEGACY_SERVER_CONNECT
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

url = "https://servicodados.ibge.gov.br/api/v3/agregados/156/periodos/1991|2000|2010/variaveis/134?localidades=N6[all]"

with requests.session() as s:
    s.mount("https://", TLSAdapter())
    s.get(url).json()
    response = s.get(url)

### Ensure the request was successful

In [4]:
if response.status_code == 200:
    # Request was successful, so we can get the data
    data = response.json()
    print("Successful request!")
else:
    # Request failed, so we can't get the data
    print(f"Request fail with status code {response.status_code}")

Successful request!


In [6]:
import sqlite3

# 1. Conexão com o Banco de Dados
conn = sqlite3.connect('path_to_your_database.db')  # Altere para o caminho do seu banco de dados
cursor = conn.cursor()

# 2. Criação da Tabela
cursor.execute('''
CREATE TABLE IF NOT EXISTS population_data (
    id INTEGER,
    year INTEGER,
    population INTEGER,
    city TEXT,
    state TEXT,
    PRIMARY KEY(id, year)
)
''')
conn.commit()

# 3. Inserção de Dados
# Aqui, 'df' é o DataFrame que você criou no seu código original.
for _, row in df.iterrows():
    cursor.execute('''
    INSERT OR REPLACE INTO population_data (id, year, population, city, state)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['id'], row['year'], row['population'], row['city'], row['state']))
conn.commit()

# 4. Verificação de Dados
cursor.execute('SELECT * FROM population_data LIMIT 5')
for row in cursor.fetchall():
    print(row)

# 5. Encerramento da Conexão
conn.close()


(1100015, 1991, 31957, "Alta Floresta D'Oeste", 'RO')
(1100015, 2000, 26481, "Alta Floresta D'Oeste", 'RO')
(1100015, 2010, 24285, "Alta Floresta D'Oeste", 'RO')
(1100023, 1991, 83365, 'Ariquemes', 'RO')
(1100023, 2000, 74383, 'Ariquemes', 'RO')


### Cleanning and transforming the data

In [5]:
data = response.json()

# Flatten data
flattened_data = []
for item in data[0]['resultados'][0]['series']:
    id = item['localidade']['id']
    name = item['localidade']['nome']
    
    # Iterate over years in the 'serie' dictionary
    for year, population in item['serie'].items():
        # Convert '...' to NaN
        if population == '...':
            population = np.nan
        flattened_data.append({'id': id, 'name': name, 'year': year, 'population': population})

# Create DataFrame
df = pd.DataFrame(flattened_data)

# Split 'name' into 'city' and 'state'
df[['city', 'state']] = df['name'].str.split(' - ', expand=True)

# Convert the 'year' and 'population' columns
df['year'] = df['year'].astype(int)
df['population'] = df['population'].fillna(-1).astype(int)
df['population'] = df['population'].replace(-1, np.nan)

# Drop the 'name' columns
df = df.drop(columns=['name'])

print(df)

# Path to the CSV file
csv_path = 'C:\\Users\\jlfen\\OneDrive\\Documentos\\JoaoKasten\\005_applied_projects\\project.censo-dataviz\\src\\data\\census_data.csv'

# Check if the CSV file already exists
if os.path.exists(csv_path):
    # Load existing data from CSV
    df_existing = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=True)
    
    # Combine new and existing data
    df_combined = pd.concat([df_existing, df], ignore_index=True)
    
    # Remove duplicates based on 'id' and 'year' columns
    df_combined.drop_duplicates(subset=['id', 'year'], inplace=True)
    
    # Save the combined DataFrame back to the CSV
    df_combined.to_csv(csv_path, index=False, encoding='utf-8-sig')
else:
    # If the file doesn't exist, just save the new data
    df.to_csv(csv_path, index=False)

            id  year  population                   city state
0      1100015  1991     31957.0  Alta Floresta D'Oeste    RO
1      1100015  2000     26481.0  Alta Floresta D'Oeste    RO
2      1100015  2010     24285.0  Alta Floresta D'Oeste    RO
3      1100023  1991     83365.0              Ariquemes    RO
4      1100023  2000     74383.0              Ariquemes    RO
...        ...   ...         ...                    ...   ...
16690  5222302  2000      4492.0          Vila Propício    GO
16691  5222302  2010      5145.0          Vila Propício    GO
16692  5300108  1991   1593455.0               Brasília    DF
16693  5300108  2000   2044211.0               Brasília    DF
16694  5300108  2010   2558865.0               Brasília    DF

[16695 rows x 5 columns]


C:\Users\jlfen\AppData\Local\Temp\ipykernel_28252\2065828726.py:38: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_existing = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=True)
C:\Users\jlfen\AppData\Local\Temp\ipykernel_28252\2065828726.py:38: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_existing = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=True)
